In [ ]:
%pip install -v --extra-index-url https://test.pypi.org/simple/ sanafe==0.0.1
import sanafe

In [ ]:
dir(sanafe)

### Running for the first time ###
First we will run SANA-FE for the first time using an example network and example architecture. This is analogous to the SANA-FE 'hello world'.

In [ ]:
# First, load an architecture description and build the chip (for simulation)
hello_arch = sanafe.load_arch("arch.yaml")
hello_chip = sanafe.SpikingHardware(hello_arch)
# Second, load the SNN from file and upload it to the chip. We pass the arch
#  as an extra argument so that SANA-FE can check the mappings are to valid
#  cores
hello_snn = sanafe.load_net("snn.yaml", hello_arch)
hello_chip.load(hello_snn)
# Third, run for 200 simulated timesteps and print a summary of the results
results = hello_chip.sim(200)
print(results)

### Architecture Description File ###
Next, we will look at an example of an SNN-based architecture in SANA-FE. Inside `arch.yaml` there is a minimal architecture, based on the diagram below. Follow the three exercises given at the top of the `arch.yaml` file to extend the design and add the dashed orange elements (shown below).
Make changes and try rerunning the cell above. Once you've made the right changes, you should see the cell pass checks!

![Example architecture](example_arch_small.png)

In [ ]:
# code to check the output of the cell above

### SNN Description File ###
Next, we will finish the SNN defined in `snn.yaml` and map it to our updated architecture. The SNN is shown in the diagram below, with pre-defined elements drawn in black. Complete the four exercises listed in the file to add the orange elements shown and complete this small SNN.

![Example SNN](example_snn_small.png)


In [ ]:
# code to check the snn description stuff

### Python interface ###
SANA-FE v2 supports building SNNs in Python, in addition to file-based inputs. The example SNN has been recreated in Python code, now try completing the four exercises again but for Python. As a reminder, these were:
#### Exercises ####
1. Define a new mapped neuron: 1.1. To do this, add another neuron to group 1 (i.e. increment the number of neurons in the group) and map neuron 1.1 to core 0.1
2. Add edges from neurons 0.0 and 0.1, both to neuron 1.1, with weights -2 & 3 respectively
3. Set the bias of neuron 0.1 to 0.5
4. Configure Group 1 to use the new compressed synapses that you defined H/W for in architecture description, instead of the uncompressed synapses used currently

In [ ]:
# Create an example SNN
in_attributes = {"threshold": 1.0, "reset": 0.0, "log_spikes": True, "log_potential": True}
out_attributes = {"threshold": 2.0, "reset": 0.0, "synapse_hw_name": "tutorial_synapse_uncompressed"}

# Create neuron groups
snn = sanafe.Network()
in_group = snn.create_neuron_group("in", 2, in_attributes)
out_group = snn.create_neuron_group("out", 1, out_attributes)

# Set neuron attributes
in_group.neurons[0].set_attributes(model_parameters={"bias": 0.2})

# Create connections
in_group.neurons[0].connect_to_neuron(out_group.neurons[0], {"weight": -1.0})

# Create mappings
arch = sanafe.load_arch("arch.yaml")
core = arch.tiles[0].cores[0]
in_group.neurons[0].map_to_core(core)
in_group.neurons[1].map_to_core(core)
out_group.neurons[0].map_to_core(core)

Once you've made all the changes, execute the code below to run your updated SNN on the updated arch.

In [ ]:
tutorial_chip = sanafe.SpikingHardware(arch)
tutorial_chip.load(snn)
results = tutorial_chip.sim(100, heartbeat=1)
print(results)

# Add code to check

Now that both an architecture and SNN have been defined, we can look at the
outputs SANA-FE can generate for these files. We use -o to specify an output
directory. First, we will just look at the run summary that SANA-FE writes
to a file, saving all output to the `tutorial` directory.

    python3 sim.py -o tutorial tutorial/arch.yaml tutorial/snn.net 10
    cat tutorial/run_summary.yaml

The file `run_summary.yaml` matches the printed summary at the end of the run.

Next, run another simulation but with the spike and potential trace flags set.
To enable spike traces, prepend (`-s`) to your run command. If the spike trace
flag is set, the simulator will record spikes for neurons with `log_spikes` set
to 1 in the SNN description. Similarly, the potential trace is set by
prepending (`-v`) to the run command. If enabled, SANA-FE logs the neuron
potentials of any voltage probes. Voltage probes are set by defining
`log_potential` to 1 for neurons in the SNN description.

In this example, two different flags are set simultaneously, i.e., (`-s -v`).
Therefore, two traces will be generated: `spikes.csv` and `potential.csv`.

    python3 sim.py -s -v -o tutorial tutorial/arch.yaml tutorial/snn.net 10
    cat tutorial/spikes.csv
    cat tutorial/potential.csv

Every line in the spike trace (`tutorial/spikes.csv`) has the format:

    <neuron>,<timestep spiked>

The potential trace (`tutorial/potential.csv`) has a column per probe and
one line per time-step i.e.:

    <neuron 0, timestep 0>,<neuron 1, timestep 0>
    <neuron 0, timestep 1>,<neuron 1, timestep 1>
    ...

Try visualizing `potential.csv` by plotting the dynamics of neurons 0.0 and
0.1 for 10 simulated timesteps, using your preferred plotting application.

Next, run the same simulation but with message and performance traces
enabled instead. Message tracing is set by prepending `-m` to the command.
This will cause the simulator to record information about spike messages sent
by hardware. Performance tracing is set by prepending `-p` to the command.
Performance traces contain more detailed per-timestep information about
activity on the simulated design. Similar to the last example, we can combine
multiple traces, using `-m -p`.

    python3 sim.py -m -p -o tutorial tutorial/arch.yaml tutorial/snn.net 10
    cat tutorial/perf.csv
    cat tutorial/messages.csv

Two different traces will be generated: `perf.csv` and `messages.csv`.
Both files have similar formats, with one line per time-step and different
columns for various statistics. Try visualizing the `fired` and `total_energy`
fields over time.